In [ ]:
import numpy as np
# from pennylane.optimize import NesterovMomentumOptimizer, SPSAOptimizer
import LHC_QML_module as lqm
from sklearn.model_selection import train_test_split
import time
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

from qiskit import QuantumCircuit
# from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler, Estimator
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator

In [ ]:
# settings to tune

seed = 123
# Features to train on
training_feature_keys = [
    "f_mass4l",
    # "f_eta4l",
    "f_Z2mass",
    "f_Z1mass",
]

save_folder = os.path.join("saved", "test-qiskit-model1g3-r5")

batch_size = 2
n_epochs = 2

train_data_size = 80
test_data_size = 80
valid_data_size = 40
total_datasize = train_data_size + test_data_size + valid_data_size
half_datasize = total_datasize // 2 # 80 signal and 80 backgrounds
# opt = NesterovMomentumOptimizer(0.1)
# opt = SPSAOptimizer(maxiter=200)

is_local_simulator = True

spsa_alpha = 0.2
spsa_gamma = 0.101
spsa_c     = 0.2
spsa_A     = 2.
spsa_a1    = 0.2
spsa_a     = spsa_a1 * (spsa_A + 1) ** spsa_alpha

In [ ]:
np.random.seed(seed)

if os.path.exists(save_folder):
    print(f"This notebook may overwrite previous results in the {save_folder} directory")
else:
    os.makedirs(save_folder)

n_inputs = len(training_feature_keys)
n_qubits = n_inputs

In [ ]:
num_layers = 5
n_qubits = 3

par_inputs = ParameterVector("input", n_qubits)
par_weights = ParameterVector("weights", num_layers * n_qubits)

qc = QuantumCircuit(3)
for i in range(n_qubits):
    qc.rx(par_inputs[i], i)

for i in range(n_qubits):
    qc.ry(par_weights[i], i)

for i in range(n_qubits):
    qc.cx(i, (i+1)%n_qubits)

for i in range(n_qubits):
    qc.rz(par_weights[i+3], i)

for i in range(n_qubits):
    qc.cx(i, (i+1)%n_qubits)

for i in range(n_qubits):
    qc.rx(par_weights[i+6], i)

for i in range(n_qubits):
    qc.cx(i, (i+1)%n_qubits)

for i in range(n_qubits):
    qc.rz(par_weights[i+9], i)

for i in range(n_qubits):
    qc.cx(i, (i+1)%n_qubits)

for i in range(n_qubits):
    qc.rz(par_weights[i+12], i)

for i in range(n_qubits):
    qc.cx(i, (i+1)%n_qubits)


obs = SparsePauliOp("XXI")

In [ ]:
qc.draw("mpl", style="clifford")

In [ ]:
def loss(prob, label):
    # print(prob)
    return -np.mean(label*np.log(prob+1e-5)+(1-label)*np.log(1-prob+1e-5))

def accuracy(pred, label):
    return np.mean(np.isclose(pred,label))

def cost(weights, features, labels):
    probs = np.array([model(weights, f) for f in features])
    return loss(probs, labels)